# VirusTotal Behavior with Microsoft Sysmon Detonation

This notebook uses the VirusTotal API to retrieve detonation information
for a file ID (the SHA256 hash of the file).

There are 5 main sections:
- Setup
- Retrieve basic information about the file ID
- Retrieve and browse denotation data for the file
- Build and view the process tree for the suspect behavior
- Generate KQL filter clauses for subsets of the data to use in Azure Sentinel queries.

In [ ]:
%pip install --upgrade msticpy[vt3]

In [110]:
REQ_PYTHON_VER=(3, 6)
REQ_MSTICPY_VER=(1, 0, 0)

from msticpy.nbtools import nbinit
nbinit.init_notebook(
    namespace=globals(),
);

In [111]:
from msticpy.sectools.vtlookupv3 import VTLookupV3
from msticpy.common.provider_settings import get_provider_settings
import nest_asyncio

nest_asyncio.apply()

# Initialize VT Client
vt_settings = get_provider_settings("TIProviders").get("VirusTotal")

vt = VTLookupV3(vt_settings.args.get("AuthKey"))

# VirusTotal Basic File Data

In [112]:
import pprint
import ipywidgets as widgets
import pandas as pd

vt_df: pd.DataFrame = None

border_layout = widgets.Layout(
    **{
        "width": "90%",
        "border": "solid gray 1px",
        "margin": "1pt",
        "padding": "5pt",
    }
)


def _ts_to_pydate(data):
    """Replace Unix timestamps in VT data with Py/pandas Timestamp."""
    for date_col in (col for col in data.columns if col.endswith("_date")):
        data[date_col] = pd.to_datetime(data[date_col], unit="s", utc=True)
    return data


def get_summary(data=None):
    data = data if data is not None else vt_df
    if data is None:
        return {"sha256": "", "meaningful_name": "", "names": "", "magic": ""}
    return data[["sha256", "meaningful_name", "names", "magic"]].iloc[0].to_dict()


def summary_html(title, summary):
    return f"""
    <h3>{title}</h3>
    <table>
    <tr>
        <td>ID</td><td>{summary.get('sha256')}</td>
    </tr>
    <tr>
        <td>Names</td><td>{summary.get('names')}</td>
    </tr>
    <tr>
        <td>File Type</td><td>{summary.get('magic')}</td>
    </tr>
    </table>
    """


def lookup_file_id(btn):
    del btn
    global vt_df
    vt_df = vt.get_object(txt_file_id.value, vt_type="file").T
    vt_df = _ts_to_pydate(vt_df)
    data_sel.options = vt_df.columns
    html_header.value = summary_html(basic_title, get_summary(vt_df))


data_sel = widgets.Select(
    description="Attribute",
    layout=widgets.Layout(height="400px")
)

data_view = widgets.Textarea(
    description="Value",
    layout=widgets.Layout(height="400px", width="60%")
)


def _display_attribute(change):
    item = change.get("new")
    data = vt_df.iloc[0][item]
    data_view.value = pprint.pformat(data)
    
data_sel.observe(_display_attribute, names="value")


txt_file_id = widgets.Text(
    description="Enter file ID (hash)",
    layout=widgets.Layout(width="70%"),
    style={"description_width": "150px"},
)

btn_lookup = widgets.Button(description="Lookup")
btn_lookup.on_click(lookup_file_id)

basic_title = "VirusTotal File hash lookup"
html_header = widgets.HTML(summary_html(
    basic_title,
    get_summary()
), layout=border_layout)

hb_file_lookup = widgets.HBox([txt_file_id, btn_lookup], layout=border_layout)
hb_vt_attribs = widgets.HBox([data_sel, data_view], layout=border_layout)
display(widgets.VBox([html_header, hb_file_lookup, hb_vt_attribs]))

# Get Microsoft Detonation Details

In [113]:
# Retrieve Behavior results
import requests
import re
from pprint import pformat

vt_root = "https://www.virustotal.com/api/v3"

file_behavior_uri = f"{vt_root}/files/{{id}}/behaviour_summary"
ms_file_behavior_uri = f"{vt_root}/file_behaviours/{{id}}_Microsoft Sysinternals"

hdr = {"headers": {"X-Apikey": vt_settings.args.get("AuthKey")}}


def get_vt_file_behavior(file_id):
    resp = requests.get(file_behavior_uri.format(id=file_id), **hdr)
    if resp.status_code == 200:
        return resp.json().get("data")


def get_vt_file_behavior_mssi(file_id):
    resp = requests.get(ms_file_behavior_uri.format(id=file_id), **hdr)
    if resp.status_code == 200:
        return resp.json().get("data", {})


mssi_behavior = get_vt_file_behavior_mssi(txt_file_id.value)
categories = mssi_behavior.get("attributes")
behavior_links = mssi_behavior.get("links")
analysis_date = pd.Timestamp(mssi_behavior["attributes"]["analysis_date"])

_CAT_PATTERNS = {
    "File": "file.*",
    "Process": "process.*|command.*|module.*",
    "Registry": "registry.*",
    "Network": ".*ips|dns.*|.*urls|ip.*|http.*",
    "System": "mutex.*|calls.*|permissions.*|text.*",
    "Other": ".*"
}


def extract_subcats(pattern, categs):
    return {cat for cat in categs if re.match(pattern, cat)}


def data_format(data_item):
    if not data_item:
        return ""
    if isinstance(data_item, list):
        if isinstance(data_item[0], dict):
            return pd.DataFrame(data_item).style.hide_index().render()
        if isinstance(data_item[0], str):
            return pd.DataFrame(pd.Series(data_item)).style.hide_index().render()
    return f"<pre>{pformat(data_item)}</pre>"
        

groupings = {}
remaining_categories = set(categories)
for name, pattern in _CAT_PATTERNS.items():
    groupings[name] = extract_subcats(pattern, remaining_categories)
    remaining_categories = remaining_categories - groupings[name]

main_tab = widgets.Tab()
child_tabs = {}
for group, sub_cats in groupings.items():

    sub_cat_tab = widgets.Tab()
    tab_content = {
        section: widgets.HTML(value=data_format(items))
        for section, items in categories.items()
        if items and section in sub_cats
    }
    sub_cat_tab.children = list(tab_content.values())
    for idx, section in enumerate(tab_content):
        sub_cat_tab.set_title(idx, section)
    child_tabs[group] = sub_cat_tab

main_tab.children = list(child_tabs.values())
for idx, group_name in enumerate(child_tabs):
    main_tab.set_title(idx, group_name)

html_title = widgets.HTML(summary_html(
    "VT Detonation Details",
    get_summary()
), layout=border_layout)
display(widgets.VBox([html_title, main_tab]))


# Display Process Tree

## First cell builds the tree

In [114]:
from copy import deepcopy, copy
import attr
from pathlib import Path


@attr.s(auto_attribs=True)
class SIProcess:

    process_id: str
    name: str
    cmd_line: str
    parent_id: int = -1
    proc_key: str = None
    parent_key: str = None
    path: str = None
    IsRoot: bool = False
    IsLeaf:bool = False
    IsBranch: bool = False
    children: list = []
    # proc_children: list = []


def create_si_proc(raw_proc):
    """Return an SIProcess Object from a raw VT proc definition."""
    # raw_proc = copy(raw_proc)
    name = raw_proc.get("name")
    raw_proc["cmd_line"] = name
    for proc in procs_created:
        if name.lower().endswith(proc):
            raw_proc["name"] = procs_created[proc]
            break
    raw_proc["proc_key"] = raw_proc["process_id"] + "|" + raw_proc["name"]
    # print(name, raw_proc.keys())
    return SIProcess(**raw_proc)

    
def extract_processes(process_data, parent=None):
    """Convert processes_tree attribute to SIProcessObjects."""
    procs = []
    for process in process_data:
        si_proc = create_si_proc(process)
        if parent:
            si_proc.parent_key = parent.proc_key
            si_proc.IsBranch = True
        else:
            si_proc.IsRoot = True
        child_procs_raw = process.get("children", [])
        if child_procs_raw:
            si_proc.children = extract_processes(child_procs_raw, parent=si_proc)
        else:
            si_proc.IsLeaf = True
            si_proc.IsBranch = False
        procs.append(si_proc)
    return procs


# Convert to DF
def procs_to_df(procs):
    """Convert the SIProcess objects to a list."""
    df_list = []
    for proc in procs:
        df_list.append(attr.asdict(proc))
        if proc.children:
            df_list.extend(procs_to_df(proc.children))
    return df_list


proc_tree_raw = deepcopy(categories["processes_tree"])
procs_created = {Path(proc).parts[-1].lower(): proc for proc in categories["processes_created"]}

si_procs = extract_processes(proc_tree_raw)
proc_tree_df = pd.DataFrame(procs_to_df(si_procs)).drop(columns="children")
proc_tree_df.head(4)

,process_id,name,cmd_line,parent_id,proc_key,parent_key,path,IsRoot,IsLeaf,IsBranch
0,3308,C:\Windows\explorer.exe,%WINDIR%\explorer.exe,-1,3308|C:\Windows\explorer.exe,None,None,True,False,False
1,3616,%SAMPLEPATH%\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe,%SAMPLEPATH%\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe,-1,3616|%SAMPLEPATH%\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe,3308|C:\Windows\explorer.exe,None,False,False,True
2,2624,C:\Windows\System32\cmd.exe,C:\Windows\System32\cmd.exe,-1,2624|C:\Windows\System32\cmd.exe,3616|%SAMPLEPATH%\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe,None,False,False,True
3,1992,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,-1,1992|C:\Windows\System32\WindowsPowerShell\v1.0\powershell.exe,2624|C:\Windows\System32\cmd.exe,None,False,True,False


In [115]:
def try_match_commandlines(command_executions, procs_cmds):
    procs_cmd = procs_cmds.copy()
    procs_cmd["cmd_line"] = np.nan
    weak_matches = 0
    for cmd in command_executions:
        for idx, row in procs_cmd.iterrows():
            # print(row["name"], cmd, row["cmd_line"], isinstance(row["cmd_line"], str))
            if (
                not isinstance(row["cmd_line"], str)
                and np.isnan(row["cmd_line"])
                and row["name"] in cmd
            ):
                # print("Found match:", row["name"], "==", cmd)
                procs_cmd.loc[idx, "cmd_line"] = cmd
                break
    for cmd in command_executions:
        for idx, row in procs_cmd.iterrows():
            # print(row["name"], cmd, row["cmd_line"], isinstance(row["cmd_line"], str))
            if (
                not isinstance(row["cmd_line"], str)
                and np.isnan(row["cmd_line"])
                and Path(row["name"]).stem.lower() in cmd.lower()
            ):
                weak_matches += 1
                # print("Found weak match:", row["name"], "~=", cmd)
                procs_cmd.loc[idx, "cmd_line"] = cmd
                break

    if weak_matches:
        print(
            f"WARNING: {weak_matches} of the {len(command_executions)} commandlines",
            "were weakly matched - some commandlines could be attributed",
            "to the wrong process.",
            end="\n"
        )
    return procs_cmd

proc_tree_cmd_df = try_match_commandlines(categories["command_executions"], proc_tree_df)

## Display the tree

In [116]:
from msticpy.sectools.proc_tree_builder import ProcSchema, _build_proc_tree


# ProcSchema
proc_schema = {
    "process_name": "name",
    "process_id": "process_id",
    "parent_id": "parent_id",
    "cmd_line": "cmd_line",
    "time_stamp": "time_stamp",
    "logon_id": "logon_id",
    "path_separator": "\\",
    "user_name": "user_name",
    "host_name_column": "host",
    "event_id_column": "event_id",
}

processes = proc_tree_cmd_df
processes["path"] = np.nan
processes.loc[processes.IsRoot, "path"] = processes[processes.IsRoot].index.astype("str")

processes["time_stamp"] = analysis_date
processes["host"] = "sandbox"
processes["logon_id"] = "na"
processes["event_id"] = "na"
processes["source_index"] = processes.index.astype("str")

proc_tree = processes.set_index("proc_key")

first_unique = proc_tree.index.duplicated()
proc_tree = proc_tree[~first_unique]
process_tree_df = _build_proc_tree(proc_tree)
process_tree_df

process_tree_df.mp_plot.process_tree(schema=ProcSchema(**proc_schema), legend_col="name")

Loading BokehJS ...

(Figure(id='1457', ...), Row(id='1571', ...))

# KQL Query Clause generator
Use this to generate filter clauses to search for the behaviors.


## Select the category of data to generate the query clause from.

In [117]:
sel_category = widgets.Select(
    description="Select a category",
    options=list(categories.keys()),
    layout=widgets.Layout(width="30%", height="200px"),
    style={"description_width": "150px"}
)
txt_values = widgets.HTML(layout=widgets.Layout(width="60%", height="300px"),)
vb_sel_category = widgets.HBox([sel_category, txt_values], layout=border_layout)


def update_cat_values(change):
    cat = change.get("new")
    data = categories.get(cat)
    txt_values.value = data_format(data)


sel_category.observe(update_cat_values, names="value")
update_cat_values({"new": sel_category.value})
display(vb_sel_category)


## Select the items to search for and create the filter

You should paste the filter into a query. You must replace "{column_name}" with the column appropriate to the table

In [120]:

def get_cat_items(cat):
    cat_items = categories.get(cat)
    if not isinstance(cat_items, list):
        return [cat_items]
    if cat == "command_executions":
        return [
            item.strip()[1:-1] if (item.strip()[0] == '"' and item.strip()[-1] == '"') else item.strip()
            for item in cat_items
        ]
    if cat == "files_dropped":
        return [item["path"] for item in cat_items]
    if cat == "ip_traffic":
        return [item["destination_ip"] for item in cat_items]
    if cat == "dns_lookups":
        items = [item["hostname"] for item in cat_items]
        for item in cat_items:
            items.extend(item["resolved_ips"])
        return items
    if not isinstance(cat_items[0], str):
        return [str(item) for item in cat_items]
    return cat_items


sel_subset = nbwidgets.SelectSubset(
    source_items=get_cat_items(sel_category.value),
    default_selected=get_cat_items(sel_category.value),
    auto_display=False
)

vb_select_items = widgets.VBox([sel_subset.layout], layout=border_layout)


KQL_TEMPLATE = "| where {{column_name}} in (\n  {values}\n)"

text_query = widgets.Textarea(
    description="KQL query filter",
    layout=widgets.Layout(width="90%", height="300px"),
)

def create_query_filter(btn):
    del btn
    text_query.value = KQL_TEMPLATE.format(
        values=',\n  '.join(f"'{item}'" for item in sel_subset.selected_values)
    )

btn_query = widgets.Button(description="Gen KQL filter")
btn_query.on_click(create_query_filter)
vb_query = widgets.VBox([btn_query, text_query], layout=border_layout)

hdr_html = widgets.HTML(f"<h3>Values for {sel_category.value} detonation category</h3")
display(widgets.VBox([hdr_html, vb_select_items, vb_query]))

In [92]:
vb_select_items.children[0]

(VBox(children=(Text(value='', description='Filter:', style=DescriptionStyle(description_width='initial')), HBox(children=(SelectMultiple(description='Source: ', layout=Layout(height='200px', width='40%'), options=('%SAMPLEPATH%\\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe" ', '%USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe  "%SAMPLEPATH%\\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe', '%USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe  "C:\\Windows\\system32\\services64.exe', 'C:\\Windows\\System32\\cmd.exe" /C choice /C Y /N /D Y /T 3 & Del "%USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe', 'C:\\Windows\\System32\\cmd.exe" /c %USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe "%SAMPLEPATH%\\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe', 'C:\\Windows\\System32\\cmd.exe" /c %USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe "C:\\Windows\\system32\\services64.exe', 'C:\\Windows\\System32\\cmd.exe" /c schtas

# TI Additional Lookups

In [32]:
ioc_txt = widgets.Text(
    description="Enter IoC to lookup",
    layout=widgets.Layout(width="70%"),
    style={"description_width": "150px"},
)

ioc_txt

Text(value='', description='Enter IoC to lookup', layout=Layout(width='70%'), style=DescriptionStyle(descripti…

In [279]:
ti = TILookup()
TILookup.browse(TILookup.result_to_df(ti.lookup_ioc(ioc_txt.value)))

Using Open PageRank. See https://www.domcop.com/openpagerank/what-is-openpagerank


e:\src\microsoft\msticpy\msticpy\sectools\tilookup.py:334: UserWarning: Could not find provider class for RiskIQ in config section RiskIQ
  f"Could not find provider class for {provider_name} "


In [36]:
categories

{'command_executions': ['"%SAMPLEPATH%\\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe" ',
  '"cmd" /c powershell -Command Add-MpPreference -ExclusionPath \'%%UserProfile%%\' & powershell -Command Add-MpPreference -ExclusionPath \'%%AppData%%\' & powershell -Command Add-MpPreference -ExclusionPath \'%%Temp%%\' & powershell -Command Add-MpPreference -ExclusionPath \'%%SystemRoot%%\' & exit',
  "powershell  -Command Add-MpPreference -ExclusionPath '%USERPROFILE% ",
  "powershell  -Command Add-MpPreference -ExclusionPath '%USERPROFILE%\\AppData\\Roaming' ",
  "powershell  -Command Add-MpPreference -ExclusionPath '%USERPROFILE%\\AppData\\Local\\Temp' ",
  "powershell  -Command Add-MpPreference -ExclusionPath 'C:\\Windows' ",
  '"C:\\Windows\\System32\\cmd.exe" /c %USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe "%SAMPLEPATH%\\03bd9a94482f180bb047626cb2f27ccf8daa0e201345480b43585580e09c311b.exe"',
  '%USERPROFILE%\\AppData\\Local\\Temp\\svchost64.exe  "%SAMPLEPATH%\\

In [34]:
vt_uri = "https://www.virustotal.com/api/v3/files/{id}/contacted_domains"

hdr = {"headers": {"X-Apikey": vt_settings.args.get("AuthKey")}}
resp = requests.get(vt_uri.format(id=txt_file_id.value), **hdr)

display(HTML("<h2>Contacted domains</h2>"))
for item in resp.json()["data"]:
    for attr in ("whois", "last_dns_records", "last_analysis_stats", "last_analysis_date"):
        print(attr, end=": ")
        if attr.endswith("_data"):
            pd.to_datetime(item["attributes"].get(attr))
        else:
            pprint.pprint(item["attributes"].get(attr))
        

whois: ('Administrative city: REDACTED FOR PRIVACY\n'
 'Administrative country: REDACTED FOR PRIVACY\n'
 'Administrative state: REDACTED FOR PRIVACY\n'
 'Create date: 2021-06-17\n'
 'Domain name: sanctam.net\n'
 'Domain registrar id: 69\n'
 'Domain registrar url: http://domainhelp.opensrs.net\n'
 'Expiry date: 2024-06-17\n'
 'Query time: 2021-06-21 15:30:16\n'
 'Registrant address: ff3a4678d9c7a906\n'
 'Registrant city: ff3a4678d9c7a906\n'
 'Registrant company: ff3a4678d9c7a906\n'
 'Registrant country: Saint Kitts and Nevis\n'
 'Registrant email: 3267309318f7846cs@\n'
 'Registrant fax: ff3a4678d9c7a906\n'
 'Registrant name: ff3a4678d9c7a906\n'
 'Registrant phone: ff3a4678d9c7a906\n'
 'Registrant state: 347e121c541b794d\n'
 'Registrant zip: ff3a4678d9c7a906\n'
 'Technical city: REDACTED FOR PRIVACY\n'
 'Technical country: REDACTED FOR PRIVACY\n'
 'Technical state: REDACTED FOR PRIVACY\n'
 'Update date: 2021-06-17')
last_dns_records: [{'ttl': 21600, 'type': 'NS', 'value': '2-nest.njalla.

In [35]:
vt_uri = "https://www.virustotal.com/api/v3/files/{id}/contacted_urls"

hdr = {"headers": {"X-Apikey": vt_settings.args.get("AuthKey")}}
resp = requests.get(vt_uri.format(id=txt_file_id.value), **hdr)

display(HTML("<h2>Contacted URLs</h2>"))
for item in resp.json()["data"]:
    for attr in ("url", "threat_names", "last_final_url", "last_analysis_stats", "last_analysis_date"):
        print(attr, end=": ")
        if attr.endswith("_data"):
            pd.to_datetime(item["attributes"].get(attr))
        else:
            pprint.pprint(item["attributes"].get(attr))
        

url: 'https://sanctam.net:58899/assets/txt/resource_url.php?type=xmrig'
threat_names: []
last_final_url: 'https://sanctam.net:58899/assets/txt/resource_url.php?type=xmrig'
last_analysis_stats: {'harmless': 74,
 'malicious': 2,
 'suspicious': 2,
 'timeout': 0,
 'undetected': 11}
last_analysis_date: 1633430508


In [72]:
resp.json()

{'meta': {'count': 1},
 'data': [{'attributes': {'html_meta': {},
    'last_final_url': 'http://www.blackievirus.com/',
    'tags': [],
    'url': 'http://www.blackievirus.com/',
    'last_analysis_date': 1615981136,
    'trackers': {},
    'has_content': False,
    'last_submission_date': 1615981136,
    'threat_names': ['C2/Generic-A'],
    'last_analysis_results': {'Tencent': {'category': 'harmless',
      'result': 'clean',
      'method': 'blacklist',
      'engine_name': 'Tencent'},
     'CMC Threat Intelligence': {'category': 'harmless',
      'result': 'clean',
      'method': 'blacklist',
      'engine_name': 'CMC Threat Intelligence'},
     'CLEAN MX': {'category': 'harmless',
      'result': 'clean',
      'method': 'blacklist',
      'engine_name': 'CLEAN MX'},
     'DNS8': {'category': 'harmless',
      'result': 'clean',
      'method': 'blacklist',
      'engine_name': 'DNS8'},
     'MalSilo': {'category': 'harmless',
      'result': 'clean',
      'method': 'blacklist',

In [41]:
beh_df = pd.json_normalize(resp.json())

os_family = entities.OSFamily.Windows if vt_df.iloc[0]["type_description"].startswith("Win32") else entities.OSFamily.Linux
def vt_proc_to_ent(proc):
    file_path = proc["name"].split()[0]
    if "\\" in file_path:
        f_dir, f_name = file_path.rsplit("\\", maxsplit=1)
    else:
        f_dir, f_name = None, file_path
    proc_entity = entities.Process(
        ProcessId=proc["process_id"],
        CommandLine=proc["name"],
        ImageFile=entities.File(Name=f_name, Directory=f_dir, OSFamily=os_family)
    )
    return proc_entity


procs = []
last_proc = None
for proc in beh_df["data.processes_tree"].iloc[0]:
    proc_entity = vt_proc_to_ent(proc)
    procs.append(proc_entity)
    if "children" in proc:
        for child in proc["children"]:
            ch_ent = vt_proc_to_ent(child)
            ch_ent.ParentProcess = proc_entity
            procs.append(ch_ent)
    last_proc = proc_entity

for proc in procs:
    display(proc)
    
    

Process(ProcessId=2140, CommandLine=%windir%\System32\svchost.exe -k WerSvcGroup, Creati...)

Process(ProcessId=2960, CommandLine=wmiadap.exe /F /T /R, CreationTimeUtc=0001-01-01 00:...)

Process(ProcessId=1488, CommandLine=%windir%\system32\wbem\wmiprvse.exe, CreationTimeUtc...)

Process(ProcessId=2656, CommandLine=%windir%\system32\DllHost.exe /Processid:{3EB3C877-1...)

Process(ProcessId=2548, CommandLine=%SAMPLEPATH%, CreationTimeUtc=0001-01-01 00:00:00, I...)

Process(ProcessId=2632, CommandLine="%windir%\System32\WScript.exe" "%windir%\HELP2.VBS"...)

Process(ProcessId=1536, CommandLine=a02e945bb7feefe470273970075487b6.exe, CreationTimeUt...)

Process(ProcessId=2212, CommandLine=C:\Windows\System32\WScript.exe C:\WINDOWS\HELP2.VBS...)

In [95]:
for proc in procs:
    if proc.ParentProcess:
        print(proc.ImageFile.Name)
    else:
        print("NP:", proc.ImageFile.Name)

NP: svchost.exe
wmiadap.exe
wmiprvse.exe
DllHost.exe
NP: %SAMPLEPATH%
WScript.exe"
NP: a02e945bb7feefe470273970075487b6.exe
WScript.exe


In [96]:
beh_df.iloc[0]["data.processes_tree"]

[{'process_id': '2140',
  'name': '%windir%\\System32\\svchost.exe -k WerSvcGroup'},
 {'process_id': '2960', 'name': 'wmiadap.exe /F /T /R'},
 {'process_id': '1488', 'name': '%windir%\\system32\\wbem\\wmiprvse.exe'},
 {'process_id': '2656',
  'name': '%windir%\\system32\\DllHost.exe /Processid:{3EB3C877-1F16-487C-9050-104DBCD66683}'},
 {'process_id': '2548',
  'name': '%SAMPLEPATH%',
  'children': [{'process_id': '2632',
    'name': '"%windir%\\System32\\WScript.exe" "%windir%\\HELP2.VBS" '}]},
 {'process_id': '1536',
  'name': 'a02e945bb7feefe470273970075487b6.exe',
  'children': [{'process_id': '2212',
    'time_offset': 1,
    'name': 'C:\\Windows\\System32\\WScript.exe C:\\WINDOWS\\HELP2.VBS'}]}]

In [51]:
beh_df.columns

Index(['data.processes_injected', 'data.processes_terminated',
       'data.mutexes_created', 'data.files_opened', 'data.processes_created',
       'data.calls_highlighted', 'data.text_highlighted',
       'data.windows_searched', 'data.modules_loaded',
       'data.registry_keys_opened', 'data.tags', 'data.processes_tree',
       'data.registry_keys_set', 'data.files_copied', 'data.verdicts',
       'data.files_written', 'data.verdict_confidence', 'data.dns_lookups',
       'data.files_dropped', 'data.command_executions', 'data.verdict_labels',
       'data.http_conversations'],
      dtype='object')

In [54]:
beh_df.iloc[0]["data.http_conversations"]

[{'url': 'http://www.blackievirus.com/', 'request_method': 'GET'}]

In [ ]:
[f_path["path"] for f_path in beh_df.iloc[0]["data.files_dropped"]]

In [ ]:
dir(entities.File())

In [60]:
beh_df.T

,0
data.processes_injected,[dwwin.exe]
data.processes_terminated,"[%windir%\System32\svchost.exe -k WerSvcGroup, 2212 - ""C:\Windows\System32\WScript.exe"" ""C:\WIND..."
data.mutexes_created,[ShimCacheMutex]
data.files_opened,"[C:\WINDOWS\system32\winsock.dll, C:\492f38c38467b97578ccf357ab305c16ec6e45e5ed85f52f4015f997d9a..."
data.processes_created,"[C:\WINDOWS\system32\drwtsn32 -p 1344 -e 228 -g, C:\Windows\System32\WScript.exe C:\WINDOWS\HELP..."
data.calls_highlighted,"[SetWindowsHookExW, GetTickCount, GetAdaptersAddresses, GetSystemMetrics]"
data.text_highlighted,"[Windows Script Host, Script:\tC:\WINDOWS\HELP2.VBS\nLine:\t7\nChar:\t1\nError:\tUnspecified err..."
data.windows_searched,[Microsoft Internet Explorer]
data.modules_loaded,"[rpcrt4.dll, shlwapi.dll, version.dll, shell32.dll, advapi32.dll, ntdll.dll, kernel32.dll, c:\wi..."
data.registry_keys_opened,"[HKLM\Software\Microsoft\Windows\CurrentVersion\Explorer\KindMap, HKLM\Software\Policies\Microso..."


In [280]:
data_sel = widgets.Select(
    description="Option",
    options=sorted(beh_df.columns),
    layout=widgets.Layout(height="200px")
)

data_view = widgets.Select(
    description="Data",
    layout=widgets.Layout(height="400px", width="60%")
)

data_detail = widgets.Textarea(
    description="Detail",
    layout=widgets.Layout(height="400px", width="60%")
)


def get_data_vals(change):
    col = change.get("new")
    return beh_df[col].iloc[0]

def display_data(change):
    col = change.get("new")
    data = beh_df[col].iloc[0]
    if not isinstance(data, list):
        data = [data]
    data_view.options = data
    
def display_detail(change):
    data = change.get("new")
    data_detail.value = pprint.pformat(data)
    
data_sel.observe(display_data, names="value")
data_view.observe(display_detail, names="value")

widgets.VBox([
    widgets.HBox([data_sel, data_view]),
    data_detail
])

NameError: name 'beh_df' is not defined

In [99]:
data_view.value

{'source': 'C:\\Users\\<USER>\\Downloads\\a02e945bb7feefe470273970075487b6.exe',
 'destination': 'C:\\WINDOWS\\IE2.EXE'}